In [126]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="white")
import os
import plotly.express as px
from plotly.subplots import make_subplots
from plotly import graph_objects as go
import warnings
warnings.filterwarnings("ignore")

Note: in this case when we refer to "true" with respect to the regime, what we mean is that we allow the controller to control the f1tenth in all experiments. Proof of this can be found in the following launch file [sim_for_rtreach_aij_experiments.launch](https://github.com/pmusau17/Platooning-F1Tenth/blob/noetic-port/src/race/launch/sim_for_rtreach_aij_experiments.launch) in lines 104-107 

```
<!--Reachability safety node that incorporates dynamic obstacles-->
    <node pkg="rtreach" unless="$(arg control)" type="reach_node_uncertain_simplex" output="screen" name="reach_node_simplex"  args="$(arg wall_time) $(arg reach_time) $(arg box_display_limit) $(arg parameter_uncertainty) $(arg algorithm_name) $(arg number_of_cars) $(arg num_obstacles) $(arg position_uncertainty) $(arg velocity_uncertainty)" required="true"/>

    <node pkg="rtreach" if="$(arg control)" type="reach_node_no_simplex" output="screen" name="reach_node_simplex"  args="$(arg wall_time) $(arg reach_time) $(arg box_display_limit) $(arg parameter_uncertainty) $(arg algorithm_name) $(arg number_of_cars) $(arg num_obstacles) $(arg position_uncertainty) $(arg velocity_uncertainty)" required="true"/>
```

if the argument **control** is true, the simplex node will not be run, and the reach_node_no_simplex will be run

In [127]:
original_dfs = []
opponents = [2,3]
obstacles = [0,5]

control = ["true","false"]
velocities = ["0.5","1.0","1.5"]
velocities_fn = {"0.5":"0.500000", "1.0":"1.000000","1.5":"1.500000"}
controllers= ["disparity_extender","pure_pursuit","e2e"]
controller_names = {"disparity_extender":"Dispartiy Extender","pure_pursuit":"Pure Pursuit","e2e":"Vision Based Network"}
file_str = "{}_{}_1.000000_{}_{}_0.0_0.0_{}.csv"
collision_str = "collisions_{}_{}_{}_{}_{}_cones.csv"
for cntrl in controllers:
    for opp in opponents:
        for obs in obstacles:
            for vel in velocities:
                for regime in control:
                    
                    
                    if(regime=="true"):
                        file_name = "nosimplex"
                    else:
                        file_name = "simplex"
                        
                    
                    fh = file_str.format(file_name,cntrl,opp,obs,velocities_fn[vel])
                    ch = collision_str.format(cntrl,opp,obs,vel,regime)
                    
                    try:
                        df = pd.read_csv(fh,header=None).tail(5) # I only want the most recently run experiments here
                        df =df.rename(columns={0:'time_str',1:'time_taken_lec',2:'time_taken_safety_controller',3:'area'})
                        df['controller'] = controller_names[cntrl]
                        df['time_taken_lec'] * 100
                        df['number_of_opponents'] = opp
                        df['obstacle_presence']= obs>0
                        df['regime'] = regime
                        df['velocity'] = vel
                        
                        df = df.drop(columns = ["time_taken_safety_controller","area"])
                        
                        if(os.path.exists(ch)):
                            c_df = pd.read_csv(ch,header=None)
                            df['collision_percentage'] = ((c_df.count()[0])/5)*100
                        else:
                            df['collision_percentage'] = 0.0
                        
                        original_dfs.append(df)
                    except:
                        continue

                        
dfs = pd.concat(original_dfs)
dfs
                    
                    


,time_str,time_taken_lec,controller,number_of_opponents,obstacle_presence,regime,velocity,collision_percentage
5,01/04/2022/18:23:04,0.876521,Dispartiy Extender,2,False,true,0.5,0.0
6,01/04/2022/18:25:14,0.864547,Dispartiy Extender,2,False,true,0.5,0.0
7,01/04/2022/18:27:24,0.876153,Dispartiy Extender,2,False,true,0.5,0.0
8,01/04/2022/18:29:35,0.875509,Dispartiy Extender,2,False,true,0.5,0.0
9,01/04/2022/18:31:43,0.889403,Dispartiy Extender,2,False,true,0.5,0.0
...,...,...,...,...,...,...,...,...
5,02/04/2022/03:07:42,0.191453,Vision Based Network,3,True,false,1.5,0.0
6,02/04/2022/03:10:51,0.187395,Vision Based Network,3,True,false,1.5,0.0
7,02/04/2022/03:14:01,0.148431,Vision Based Network,3,True,false,1.5,0.0
8,02/04/2022/03:17:13,0.235294,Vision Based Network,3,True,false,1.5,0.0


In [128]:
dfs[dfs['collision_percentage'] > 0]

,time_str,time_taken_lec,controller,number_of_opponents,obstacle_presence,regime,velocity,collision_percentage
5,01/04/2022/18:33:35,0.178045,Dispartiy Extender,2,False,false,0.5,100.0
6,01/04/2022/18:35:26,0.173962,Dispartiy Extender,2,False,false,0.5,100.0
7,01/04/2022/18:37:19,0.179240,Dispartiy Extender,2,False,false,0.5,100.0
8,01/04/2022/18:39:09,0.175054,Dispartiy Extender,2,False,false,0.5,100.0
9,01/04/2022/18:41:00,0.174151,Dispartiy Extender,2,False,false,0.5,100.0
...,...,...,...,...,...,...,...,...
5,02/04/2022/03:01:25,0.171429,Vision Based Network,3,True,true,1.5,100.0
6,02/04/2022/03:02:10,0.100000,Vision Based Network,3,True,true,1.5,100.0
7,02/04/2022/03:02:55,0.144737,Vision Based Network,3,True,true,1.5,100.0
8,02/04/2022/03:03:39,0.062500,Vision Based Network,3,True,true,1.5,100.0


### Safety Analysis

Summary of all of experiments without the use of the simplex architechture, this gives us a rough sense of the performance of each controller.

In [149]:
sa = dfs[dfs["regime"]=="true"]
sa['time_taken_lec'] = sa['time_taken_lec'] * 100
#sa = dfs.drop(columns=["regime"])
res = sa.groupby(["controller","number_of_opponents","obstacle_presence","velocity"]).agg(["mean","std"]).reset_index()

In [150]:
table1 = res[res["obstacle_presence"]==True].drop(columns=["obstacle_presence"])
table1 = round(table1[table1.columns[:-1]],2)
table1 =table1.set_index("controller")
print(table1.to_latex())

\begin{tabular}{lrlrrr}
\toprule
{} & number\_of\_opponents & velocity & \multicolumn{2}{l}{time\_taken\_lec} & collision\_percentage \\
{} &           mean &    std &                 mean \\
controller           &                     &          &                &        &                      \\
\midrule
Dispartiy Extender   &                   2 &      0.5 &          87.53 &   0.50 &                  0.0 \\
Dispartiy Extender   &                   2 &      1.0 &          71.02 &   1.21 &                  0.0 \\
Dispartiy Extender   &                   2 &      1.5 &          61.61 &  11.26 &                 60.0 \\
Dispartiy Extender   &                   3 &      0.5 &          90.99 &   0.43 &                  0.0 \\
Dispartiy Extender   &                   3 &      1.0 &          63.56 &   1.06 &                  0.0 \\
Dispartiy Extender   &                   3 &      1.5 &          65.11 &   0.79 &                  0.0 \\
Pure Pursuit         &                   2 &      0.5 &  

In [153]:
table1 = res[res["obstacle_presence"]==False].drop(columns=["obstacle_presence"])
table1 = round(table1[table1.columns[:-1]],2)
table1 = table1[table1.columns[3:]]
#table1 =table1.set_index("time_taken_lec")
print(table1.to_latex())

\begin{tabular}{lrrr}
\toprule
{} & \multicolumn{2}{l}{time\_taken\_lec} & collision\_percentage \\
{} &           mean &    std &                 mean \\
\midrule
0  &          87.64 &   0.88 &                  0.0 \\
1  &          69.21 &  13.32 &                 20.0 \\
2  &          71.71 &   1.27 &                 20.0 \\
6  &          92.17 &   0.62 &                  0.0 \\
7  &          65.84 &   1.58 &                  0.0 \\
8  &          66.89 &   1.16 &                 20.0 \\
12 &         100.00 &   0.00 &                  0.0 \\
13 &          90.43 &  11.93 &                 20.0 \\
14 &          94.85 &   0.33 &                  0.0 \\
18 &         100.00 &   0.00 &                  0.0 \\
19 &          12.14 &  12.19 &                100.0 \\
20 &          73.27 &  34.94 &                 40.0 \\
24 &          95.16 &   5.24 &                 20.0 \\
25 &          16.85 &   6.09 &                100.0 \\
26 &          24.53 &   2.26 &                100.0 \\
30 &       

In [141]:
table1 = res[res["obstacle_presence"]==False].drop(columns=["obstacle_presence"])
table1 = round(table1[table1.columns[:-1]],2)

### Simplex

In [133]:
simplex = dfs[dfs["regime"]=="false"]
simplex.groupby(["controller","number_of_opponents","obstacle_presence","velocity"]).agg(["mean","std"])

time_taken_lec  \
                                                                              mean   
controller           number_of_opponents obstacle_presence velocity                  
Dispartiy Extender   2                   False             0.5            0.176090   
                                                           1.0            0.247274   
                                                           1.5            0.158714   
                                         True              0.5            0.174709   
                                                           1.0            0.251083   
                                                           1.5            0.157594   
                     3                   False             0.5            0.715132   
                                                           1.0            0.211217   
                                                           1.5            0.133934   
                                         True              0.5            0.746348   
                                                           1.0            0.206900   
                                                           1.5            0.169297   
Pure Pursuit         2                   False             0.5            1.000000   
                                                           1.0            0.883302   
                                                           1.5            0.599391   
                                         True              0.5            0.939474   
                                                           1.0            0.743729   
                                                           1.5            0.625160   
                     3                   False             0.5            1.000000   
                                                           1.0            0.672382   
                                                           1.5            0.507657   
                                         True              0.5            0.941669   
                                                           1.0            0.605043   
                                                           1.5            0.210414   
Vision Based Network 2                   False             0.5            0.917066   
                                                           1.0            0.433616   
                                                           1.5            0.298754   
                                         True              0.5            0.802429   
                                                           1.0            0.477446   
                                                           1.5            0.278882   
                     3                   False             0.5            0.977840   
                                                           1.0            0.406871   
                                                           1.5            0.382537   
                                         True              0.5            0.930162   
                                                           1.0            0.336649   
                                                           1.5            0.188259   

                                                                               \
                                                                          std   
controller           number_of_opponents obstacle_presence velocity             
Dispartiy Extender   2                   False             0.5       0.002403   
                                                           1.0       0.009674   
                                                           1.5       0.005560   
                                         True              0.5       0.001057   
                                                           1.0       0.007225   
                                                           1.5   

###  No Simplex

In [82]:
no_simplex = dfs[dfs["regime"]=="true"]
no_simplex.groupby(["controller","number_of_opponents","obstacle_presence","velocity"]).agg(["mean","std"])

time_taken_lec  \
                                                                            mean   
controller         number_of_opponents obstacle_presence velocity                  
disparity_extender 2                   False             0.5            0.876427   
                                                         1.0            0.692064   
                                                         1.5            0.717144   
                                       True              0.5            0.875328   
                                                         1.0            0.710232   
                                                         1.5            0.616113   
                   3                   False             0.5            0.921718   
                                                         1.0            0.658406   
                                                         1.5            0.668865   
                                       True              0.5            0.909887   
                                                         1.0            0.635607   
                                                         1.5            0.651068   
e2e                2                   False             0.5            0.951636   
                                                         1.0            0.168498   
                                                         1.5            0.245323   
                                       True              0.5            0.953581   
                                                         1.0            0.751367   
                                                         1.5            0.624695   
                   3                   False             0.5            0.997144   
                                                         1.0            0.359900   
                                                         1.5            0.110916   
                                       True              0.5            0.974949   
                                                         1.0            0.371350   
                                                         1.5            0.159622   
pure_pursuit       2                   False             0.5            1.000000   
                                                         1.0            0.904349   
                                                         1.5            0.948548   
                                       True              0.5            0.965559   
                                                         1.0            0.723505   
                                                         1.5            0.828641   
                   3                   False             0.5            1.000000   
                                                         1.0            0.121434   
                                                         1.5            0.732695   
                                       True              0.5            0.967915   
                                                         1.0            0.078915   
                                                         1.5            0.338553   

                                                                             \
                                                                        std   
controller         number_of_opponents obstacle_presence velocity             
disparity_extender 2                   False             0.5       0.008810   
                                                         1.0       0.133244   
                                                         1.5       0.012697   
                                       True              0.5       0.004999   
                                                         1.0       0.012094   
                                                         1.5       0.112636   
                   3                   False             0.5       0.006190   

### Safety Analysis with Simplex

In [146]:
sa = dfs[dfs["regime"]=="false"]
sa['time_taken_lec'] = sa['time_taken_lec'] * 100
#sa = dfs.drop(columns=["regime"])
res = sa.groupby(["controller","number_of_opponents","obstacle_presence","velocity"]).agg(["mean","std"]).reset_index()
res

controller number_of_opponents obstacle_presence velocity  \
                                                                          
0     Dispartiy Extender                   2             False      0.5   
1     Dispartiy Extender                   2             False      1.0   
2     Dispartiy Extender                   2             False      1.5   
3     Dispartiy Extender                   2              True      0.5   
4     Dispartiy Extender                   2              True      1.0   
5     Dispartiy Extender                   2              True      1.5   
6     Dispartiy Extender                   3             False      0.5   
7     Dispartiy Extender                   3             False      1.0   
8     Dispartiy Extender                   3             False      1.5   
9     Dispartiy Extender                   3              True      0.5   
10    Dispartiy Extender                   3              True      1.0   
11    Dispartiy Extender                   3              True      1.5   
12          Pure Pursuit                   2             False      0.5   
13          Pure Pursuit                   2             False      1.0   
14          Pure Pursuit                   2             False      1.5   
15          Pure Pursuit                   2              True      0.5   
16          Pure Pursuit                   2              True      1.0   
17          Pure Pursuit                   2              True      1.5   
18          Pure Pursuit                   3             False      0.5   
19          Pure Pursuit                   3             False      1.0   
20          Pure Pursuit                   3             False      1.5   
21          Pure Pursuit                   3              True      0.5   
22          Pure Pursuit                   3              True      1.0   
23          Pure Pursuit                   3              True      1.5   
24  Vision Based Network                   2             False      0.5   
25  Vision Based Network                   2             False      1.0   
26  Vision Based Network                   2             False      1.5   
27  Vision Based Network                   2              True      0.5   
28  Vision Based Network                   2              True      1.0   
29  Vision Based Network                   2              True      1.5   
30  Vision Based Network                   3             False      0.5   
31  Vision Based Network                   3             False      1.0   
32  Vision Based Network                   3             False      1.5   
33  Vision Based Network                   3              True      0.5   
34  Vision Based Network                   3              True      1.0   
35  Vision Based Network                   3              True      1.5   

   time_taken_lec            collision_percentage       
             mean        std                 mean  std  
0        17.60904   0.240343                100.0  0.0  
1        24.72738   0.967352                  0.0  0.0  
2        15.87136   0.556047                  0.0  0.0  
3        17.47088   0.105686                100.0  0.0  
4        25.10832   0.722466                  0.0  0.0  
5        15.75938   0.401938                  0.0  0.0  
6        71.51322   5.384946                  0.0  0.0  
7        21.12168   0.745534                  0.0  0.0  
8        13.39336   0.274313                  0.0  0.0  
9        74.63482   3.483647                  0.0  0.0  
10       20.69004   0.284459                  0.0  0.0  
11       16.92970   7.947093                  0.0  0.0  
12      100.00000   0.000000                  0.0  0.0  
13       88.33020   1.455864                  0.0  0.0  
14       59.93912   0.539362                  0.0  0.0  
15       93.94738   0.113508                  0.0  0.0  
16       74.37294   1.127115                  0.0  0.0  
17       62.51598   1.952686                  0.0  0.0  
18      100.00000   0.0

In [147]:
table1 = res[res["obstacle_presence"]==True].drop(columns=["obstacle_presence"])
table1 = round(table1[table1.columns[:-1]],2)
table1 =table1.set_index("controller")
print(table1.to_latex())

\begin{tabular}{lrlrrr}
\toprule
{} & number\_of\_opponents & velocity & \multicolumn{2}{l}{time\_taken\_lec} & collision\_percentage \\
{} &           mean &    std &                 mean \\
controller           &                     &          &                &        &                      \\
\midrule
Dispartiy Extender   &                   2 &      0.5 &          17.47 &   0.11 &                100.0 \\
Dispartiy Extender   &                   2 &      1.0 &          25.11 &   0.72 &                  0.0 \\
Dispartiy Extender   &                   2 &      1.5 &          15.76 &   0.40 &                  0.0 \\
Dispartiy Extender   &                   3 &      0.5 &          74.63 &   3.48 &                  0.0 \\
Dispartiy Extender   &                   3 &      1.0 &          20.69 &   0.28 &                  0.0 \\
Dispartiy Extender   &                   3 &      1.5 &          16.93 &   7.95 &                  0.0 \\
Pure Pursuit         &                   2 &      0.5 &  

In [148]:
table1 = res[res["obstacle_presence"]==False].drop(columns=["obstacle_presence"])
table1 = round(table1[table1.columns[:-1]],2)
table1 = table1[table1.columns[3:]]
print(table1.to_latex())

\begin{tabular}{lrrr}
\toprule
{} & \multicolumn{2}{l}{time\_taken\_lec} & collision\_percentage \\
{} &           mean &    std &                 mean \\
\midrule
0  &          17.61 &   0.24 &                100.0 \\
1  &          24.73 &   0.97 &                  0.0 \\
2  &          15.87 &   0.56 &                  0.0 \\
6  &          71.51 &   5.38 &                  0.0 \\
7  &          21.12 &   0.75 &                  0.0 \\
8  &          13.39 &   0.27 &                  0.0 \\
12 &         100.00 &   0.00 &                  0.0 \\
13 &          88.33 &   1.46 &                  0.0 \\
14 &          59.94 &   0.54 &                  0.0 \\
18 &         100.00 &   0.00 &                  0.0 \\
19 &          67.24 &  18.35 &                  0.0 \\
20 &          50.77 &   0.88 &                  0.0 \\
24 &          91.71 &   4.68 &                  0.0 \\
25 &          43.36 &  13.78 &                 40.0 \\
26 &          29.88 &   6.18 &                 60.0 \\
30 &       